# Hazard map

In [1]:
# pip install grass

In [ ]:
# pip install grass-session

In [14]:
import os
import subprocess
import sys

# define GRASS data settings (adapt to your needs)
grassdata = "/home/mneteler/grassdata/"
location = "nc_spm_08_grass7"
mapset = "user1"

# Python path: we ask GRASS GIS where its Python packages are
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS Python bindings
#import grass.script as gs
import grass.script.setup as gsetup
#
## Start GRASS Session
#session = gsetup.init(grassdata, location, mapset)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [12]:
from grass_session import Session

RuntimeError: Cannot find GRASS GIS start script: 'grass', set the right one using the GRASSBIN environm. variable

In [5]:
from grass_session import Session
#import grass.script as gs

RuntimeError: Cannot find GRASS GIS start script: 'grass', set the right one using the GRASSBIN environm. variable

In [ ]:
# buffer


distribution_raster = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f02_raster_10m_collection\aoi_osm_2024_10m_b4.tif"



# Set the GRASS GIS environment variables if necessary
# gs.gisenv()

# Define the parameters
params = {
    '-z': True,
    'GRASS_RASTER_FORMAT_META': '',
    'GRASS_RASTER_FORMAT_OPT': '',
    'GRASS_REGION_CELLSIZE_PARAMETER': 0,
    'GRASS_REGION_PARAMETER': None,
    'distances': '100,200,300',  # Buffer distances
    'input': r'N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f02_raster_10m_collection/aoi_osm_2024_10m_b4.tif',
    'output': 'TEMPORARY_OUTPUT',  # or specify a path for the output
    'units': 0  # Assuming 0 is for map units (meters)
}

# Run the buffer function on the raster
# This calls r.buffer with the specified parameters
gs.run_command('r.buffer', **params)

# You can check the output using GRASS GIS commands if needed
# For example, you can use gdal_translate to export the result or inspect it within GRASS
